Function that returns 2-dimensional boolean array, random or ordered. Each boolean digit corresponds to up- and down-spin.

**initialize** (Length, initial_state="random")

In [ ]:
function initialize(Length, initial_state="random")

    if initial_state == "random"
        Config = rand(Bool, Length,Length)
    elseif initial_state == "ordered"
        Config = ones(Bool, Length,Length)
    end

    return Config
end

Functions that make spin configuration undergo certain time of stochastic dynamics.

**evolution_rs** (Config, Temperature, Time): 	*Metropolis, random-site flipping*

**evolution_tw** (Config, Temperature, Time):	*Metropolis, typewriter flipping*

Function inputs

>**Config**:	2-dimensional boolean array of spin
>
>**Temperature**:	temperature of the stochastic dynamics
>
>**Time**:	duration of the dynamics. $\text{Time} \times \text{Length}^2$ spin-flipping attempts will be made

---------------------------------------

Functions that return a vector of sampled magnetization and energy data. Each function has distinct Markov chain.

**stream_rs** (Config, Temperature, num_sample, num_sample_burn, period_sample): *Metropolis, random-site flipping*

**stream_tw** (Config, Temperature, num_sample, num_sample_burn, period_sample): *Metropolis, typewriter flipping*

**stream_glauber** (Config, Temperature, num_sample, num_sample_burn, period_sample):   *Glauber, random-site flipping*

**stream_glauber_tw** (Config, Temperature, num_sample, num_sample_burn, period_sample):	*Glauber, typewriter flipping*

Function inputs

>**Config**:	2-dimensional boolean array of spin
>
>**Temperature**:	temperature of the stochastic dynamics
>
>**num_sample**:	number of sample physical properties
>
>**num_sample_burn**:	After this time the stream of sample is taken
>
>**period_sample**:	time between each sample

In [ ]:
function dynamics_rs(Config, Temperature, Time)
    Length = size(Config)[1]

    for time = 1:Time
        for iteration = 1:Length^2
            x = mod(rand(Int8),Length) + 1
            y = mod(rand(Int8),Length) + 1

            spin_site = Config[x,y]
            E = 4
            if Config[mod(x,Length)+1,y] == spin_site
                E -= 2
            end
            if Config[mod(x-2,Length)+1,y] == spin_site
                E -= 2
            end
            if Config[x,mod(y,Length)+1] == spin_site
                E -= 2
            end
            if Config[x,mod(y-2,Length)+1] == spin_site
                E -= 2
            end
            
            if E > 0
                Config[x,y] = !Config[x,y] # Flip
            elseif rand(Float64) < exp(2*E/Temperature)
                Config[x,y] = !Config[x,y] # Flip
            end
        end
    end
end

In [ ]:
function dynamics_tw(Config, Temperature, Time)
    Length = size(Config)[1]

    for time = 1:Time
        for x = 1:Length
            for y = 1:Length
                spin_site = Config[x,y]
                E = 4
                if Config[mod(x,Length)+1,y] == spin_site
                    E -= 2
                end
                if Config[mod(x-2,Length)+1,y] == spin_site
                    E -= 2
                end
                if Config[x,mod(y,Length)+1] == spin_site
                    E -= 2
                end
                if Config[x,mod(y-2,Length)+1] == spin_site
                    E -= 2
                end

                if E > 0
                    Config[x,y] = !Config[x,y] # Flip
                elseif rand(Float64) < exp(2*E/Temperature)
                    Config[x,y] = !Config[x,y] # Flip
                end
            end
        end
    end
end

In [ ]:
function stream_rs(Config, Temperature, num_sample, num_sample_burn, period_sample)
    Length = size(Config)[1]

    vec_magnetization = zeros(num_sample)
    vec_energy = zeros(num_sample)

    for time = 1:num_sample_burn
        for iteration = 1:Length^2
            x = mod(rand(Int8),Length) + 1
            y = mod(rand(Int8),Length) + 1

            spin_site = Config[x,y]
            E = 4
            if Config[mod(x,Length)+1,y] == spin_site
                E -= 2
            end
            if Config[mod(x-2,Length)+1,y] == spin_site
                E -= 2
            end
            if Config[x,mod(y,Length)+1] == spin_site
                E -= 2
            end
            if Config[x,mod(y-2,Length)+1] == spin_site
                E -= 2
            end
            
            if E > 0
                Config[x,y] = !Config[x,y] # Flip
            elseif rand(Float64) < exp(2*E/Temperature)
                Config[x,y] = !Config[x,y] # Flip
            end
        end
    end
    magnetization = 0
    magnetization2 = 0
    for time = 1:num_sample
        for iteration = 1:Length^2*period_sample
            x = mod(rand(Int8),Length) + 1
            y = mod(rand(Int8),Length) + 1

            spin_site = Config[x,y]
            E = 4
            if Config[mod(x,Length)+1,y] == spin_site
                E -= 2
            end
            if Config[mod(x-2,Length)+1,y] == spin_site
                E -= 2
            end
            if Config[x,mod(y,Length)+1] == spin_site
                E -= 2
            end
            if Config[x,mod(y-2,Length)+1] == spin_site
                E -= 2
            end
            
            if E > 0
                Config[x,y] = !Config[x,y] # Flip
            elseif rand(Float64) < exp(2*E/Temperature)
                Config[x,y] = !Config[x,y] # Flip
            end
        end

        vec_magnetization[time] = -Length^2 + 2*sum(Config)
        vec_energy[time] = 4*Length^2 - 2*sum((Config.==circshift(Config, (-1,0))) + (Config.==circshift(Config, (1,0))) + (Config.==circshift(Config, (0,-1))) + (Config.==circshift(Config, (0,1))))
    end
    
    return vec_magnetization, vec_energy
end

In [ ]:
function stream_tw(Config, Temperature, num_sample, num_sample_burn, period_sample)
    Length = size(Config)[1]

    vec_magnetization = zeros(num_sample)
    vec_energy = zeros(num_sample)

    for time = 1:num_sample_burn
        for x = 1:Length
            for y = 1:Length
                spin_site = Config[x,y]
                E = 4
                if Config[mod(x,Length)+1,y] == spin_site
                    E -= 2
                end
                if Config[mod(x-2,Length)+1,y] == spin_site
                    E -= 2
                end
                if Config[x,mod(y,Length)+1] == spin_site
                    E -= 2
                end
                if Config[x,mod(y-2,Length)+1] == spin_site
                    E -= 2
                end

                if E > 0
                    Config[x,y] = !Config[x,y] # Flip
                elseif rand(Float64) < exp(2*E/Temperature)
                    Config[x,y] = !Config[x,y] # Flip
                end
            end
        end
    end
    magnetization = 0
    magnetization2 = 0
    for time = 1:num_sample
        for iteration = 1:period_sample
            for x = 1:Length
                for y = 1:Length
                    spin_site = Config[x,y]
                    E = 4
                    if Config[mod(x,Length)+1,y] == spin_site
                        E -= 2
                    end
                    if Config[mod(x-2,Length)+1,y] == spin_site
                        E -= 2
                    end
                    if Config[x,mod(y,Length)+1] == spin_site
                        E -= 2
                    end
                    if Config[x,mod(y-2,Length)+1] == spin_site
                        E -= 2
                    end

                    if E > 0
                        Config[x,y] = !Config[x,y] # Flip
                    elseif rand(Float64) < exp(2*E/Temperature)
                        Config[x,y] = !Config[x,y] # Flip
                    end
                end
            end
        end

        vec_magnetization[time] = -Length^2 + 2*sum(Config)
        vec_energy[time] = 4*Length^2 - 2*sum((Config.==circshift(Config, (-1,0))) + (Config.==circshift(Config, (1,0))) + (Config.==circshift(Config, (0,-1))) + (Config.==circshift(Config, (0,1))))
    end
    
    return vec_magnetization, vec_energy
end

In [ ]:
function stream_glauber(Config, Temperature, num_sample, num_sample_burn, period_sample)
    Length = size(Config)[1]

    vec_magnetization = zeros(num_sample)
    vec_energy = zeros(num_sample)

    for time = 1:num_sample_burn
        for iteration = 1:Length^2
            x = mod(rand(Int8),Length) + 1
            y = mod(rand(Int8),Length) + 1

            x = mod(rand(Int8),Length) + 1
            y = mod(rand(Int8),Length) + 1

            field =  -4 + 2*(Config[mod(x,Length)+1,y] + Config[mod(x-2,Length)+1,y] + Config[x,mod(y,Length)+1] + Config[x,mod(y-2,Length)+1])
            
            if rand(Float64) < 1 /(1 + exp(2*field/Temperature))
                Config[x,y] = 0
            else
                Config[x,y] = 1
            end
        end
    end
    magnetization = 0
    magnetization2 = 0
    for time = 1:num_sample
        for iteration = 1:Length^2*period_sample
            x = mod(rand(Int8),Length) + 1
            y = mod(rand(Int8),Length) + 1

            field =  -4 + 2*(Config[mod(x,Length)+1,y] + Config[mod(x-2,Length)+1,y] + Config[x,mod(y,Length)+1] + Config[x,mod(y-2,Length)+1])
            
            if rand(Float64) < 1 /(1 + exp(2*field/Temperature))
                Config[x,y] = 0
            else
                Config[x,y] = 1
            end
        end

        vec_magnetization[time] = -Length^2 + 2*sum(Config)
        vec_energy[time] = 4*Length^2 - 2*sum((Config.==circshift(Config, (-1,0))) + (Config.==circshift(Config, (1,0))) + (Config.==circshift(Config, (0,-1))) + (Config.==circshift(Config, (0,1))))
    end
    
    return vec_magnetization, vec_energy
end

In [ ]:
function stream_glauber_tw(Config, Temperature, num_sample, num_sample_burn, period_sample)
    Length = size(Config)[1]

    vec_magnetization = zeros(num_sample)
    vec_energy = zeros(num_sample)

    for time = 1:num_sample_burn
        for x = 1:Length
            for y = 1:Length
                x = mod(rand(Int8),Length) + 1
                y = mod(rand(Int8),Length) + 1

                field =  -4 + 2*(Config[mod(x,Length)+1,y] + Config[mod(x-2,Length)+1,y] + Config[x,mod(y,Length)+1] + Config[x,mod(y-2,Length)+1])

                if rand(Float64) < 1 /(1 + exp(2*field/Temperature))
                    Config[x,y] = 0
                else
                    Config[x,y] = 1
                end
            end
        end
    end
    magnetization = 0
    magnetization2 = 0
    for time = 1:num_sample
        for iteration = 1:period_sample
            for x = 1:Length
                for y = 1:Length
                    x = mod(rand(Int8),Length) + 1
                    y = mod(rand(Int8),Length) + 1

                    field =  -4 + 2*(Config[mod(x,Length)+1,y] + Config[mod(x-2,Length)+1,y] + Config[x,mod(y,Length)+1] + Config[x,mod(y-2,Length)+1])

                    if rand(Float64) < 1 /(1 + exp(2*field/Temperature))
                        Config[x,y] = 0
                    else
                        Config[x,y] = 1
                    end
                end
            end
        end

        vec_magnetization[time] = -Length^2 + 2*sum(Config)
        vec_energy[time] = 4*Length^2 - 2*sum((Config.==circshift(Config, (-1,0))) + (Config.==circshift(Config, (1,0))) + (Config.==circshift(Config, (0,-1))) + (Config.==circshift(Config, (0,1))))
    end
    
    return vec_magnetization, vec_energy
end

Functions related with autocorrelation.

**autocorrelation** (data, k): 	*k-step autocorrelation of data*

**autocorrelation_function** (data, kmax):	*Returns 1- to kmax-step autocorrelation of data*

**autocorrelation_time** (data, kmax):	*Returns autocorrelation time, which is the first time that autocorrelation is less than exp(-1)*

In [ ]:
function autocorrelation(data, k)
    n = length(data)
    mu = mean(data)
    variance = var(data)
    
    corr = 0
    for t = 1:(n - k)
        corr += (data[t] - mu) * (data[t + k] - mu)
    end
    
    return corr / (variance * (n - k))
end
        
function autocorrelation_function(data, kmax)
    corr = zeros(kmax)
    for k = 0:kmax-1
        corr[k+1] = autocorrelation(data, k)
    end
    return corr
end
    
function autocorrelation_time(data, kmax)
    corr = zeros(kmax)
    for k = 0:kmax-1
        corr[k+1] = autocorrelation(data, k)
    end
    if length(findall(corr.<exp(-1))) == 0
        return Inf
    end
    t2 = findall(corr.<exp(-1))[1]
    t1 = t2 -1
    return (t2*(corr[t1]-exp(-1)) + t1*(-corr[t2]+exp(-1))) /(corr[t1]-corr[t2])
end